# Assignment 2
#### 02806 - Social data analysis and visualization 

_____
_____
_____
_____
_____

## Part 1 - Questions to text and lectures

***Exercise 1.1***

*What is the Oxford English Dictionary's defintion of a narrative?*

***Answer:*** 

*“An account of a series of events, facts, etc., given in order and with the establishing of connections between them.”*

____

***Exercise 1.2***

*What is your favorite visualization among the examples in section 3? Explain why in a few words.*

***Answer:***

*Our favorite is 3.2 - 'Budget Forecasts'. The idea of single-frame interactivity is excellent, since it compacts the visualisation, while still given a great deal of information. It also has a nice tacit tutorial, allowing users to participate, as if they are a part of exploring the visualisation. The added mouse-over (tooltip) details augments this compact graph with enough information - when you need it.*

_____

***Exercise 1.3***

*What's the point of Figure 7?*

***Answer:*** *The point with the figure is that a lot of different genres of presentation all have som ordering of the data element, and an interactive element of the data visualisation. However, most of them actually fail in (3) giving the user a tutorial on how to use it, or (4) don't use enough messaging to clearly narrate the story*
___

***Exercise 1.4***

*Use Figure 7 to find the most common design choice within each category for the Visual narrative and Narrative structure (the categories within visual narrative are 'visual structuring', 'highlighting', etc).*

***Answer:***

*The most common design choices in each category is:*

- **Visual narrative**
    - *Visual structuring*
        - *Consistent Visual Platform*
    - *Highlighting*
        - *Feature distinction*
    - *Transition guidance*
        - *Object continuity*
- **Narrative structure***
    - *Ordering*
        - *User directed path*
    - *Interactivity*
        - *Filtering / Selection / Search*
    - *Messaging*
        - *Captions / Headlines*
        
_____

***Exercise 1.5*** 

*Check out Figure 8 and section 4.3. What is your favorite genre of narrative visualization? Why? What is your least favorite genre? Why?*

***Answer:***

*Our favorite genre is Partitioned Poster. A poster can carry a wealth of information, including charts and legends, and can partition the information and present it intuitively. The vivid illustrations are full of fun, and the process of getting the details from the above is very inspiring. My least favorite is Flow Chart for it is boring and unintuitive. Furthermore, flow chart can't reveal changes in time. It will inculcate readers too strongly in the logic of the matter which may fail to combine with some details.*

_____

***Exercise 1.6***

*What are the three key elements to keep in mind when you design an explanatory visualization?*

***Answer:*** *There are 3 key elements:*

- *Start with the question (What is the question that you want to answer and communicate?)*
- *Allow exploration (Let the users investigate the data. Therefore something like D3, or even Tableau can be used)*
- *Know you readers (Make sure that you are not dumbing something down for instance to peers. Or making it overly complicated when it needs to be cross-disciplinary)*
____



***Exercise 1.7***

*In the video I talk about (1) overview first, (2) zoom and filter, (3) details on demand.*
- *Go online and find a visualization that follows these principles (don't use one from the video).*
- *Explain how it does achieves (1)-(3). It might be useful to use screenshots to illustrate your explanation.*

***Answer:***
- *Online we found [this link](http://manpopex.us/)*
- *(1) First you get a good overview of dynamic population of Manhattan. (2) Then you can zoom in and filter the data that you need, for instance, we can select one district, instead of the 12 districts highlighted. (3) Then we can specifically select data that happens on each day, to see how much it changes over the day*
_________

***Exercise 1.8***

*Explain in your own words: How is explanatory data analysis different from exploratory data analysis?*

***Answer:*** *Exploratory data analysis is where you are looking for correlations and interesting stories that the data has to tell you. This is opposed to the explanatory data analysis, where you've found the interesting story that the data has to show, and you've spent time on visualising that story, so that others may see it as well.*
_____

## Part 2: Random forest and weather

The aim here is to recreate the work you did in Part 1-3 of the Week 7 lecture. I've phrased things differently relative to the exercise to make the purpose more clear.

This exercise is divided into two parts:

- **Part 2A** - Random forest binary classification
- **Part 2B** - Info from weather features
_____

### Part 2A: Random forest binary classification

* Using the and instructions and material from Week 7, build a *random forest* classifier to distinguish between two types (you choose) of crime using on spatio-temporal (where/when) features of data describing the two crimes. When you're done, you should be able to give the classifier a place and a time, and it should tell you which of the two  types of crime happened there.
  - Explain about your choices for training/test data, features, and encoding. (You decide how to present your results, but here are some example topics to consider: Did you balance the training data? What are the pros/cons of balancing? Do you think your model is overfitting? Did you choose to do cross-validation? Which specific features did you end up using? Why? Which features (if any) did you one-hot encode? Why ... or why not?))
  - Report accuracy. Discuss the model performance.

***We start off by importing pandas, numpy, and the policing dataset***

In [32]:
import numpy as np
import pandas as pd

df_all = pd.read_csv('../rawdata/Police_Department_Incident_Reports__Historical_2003_to_May_2018.csv')

***We do some filtering of the data, so that we only have the `CATEGORY`, `DAYOFWEEK`, `TIME`, `PDDISTRICT`, AND `MONTH`***

In [33]:
# We chose the columns of interest
df_all = df_all.loc[:, ["Category","DayOfWeek","Date","Time","PdDistrict"]]

# We create a dataset for prostitution
df_pro = df_all[df_all['Category'] == 'PROSTITUTION']
# We create a dataset for vandalism
df_van = df_all[df_all['Category'] == 'VANDALISM']
# We sample 15000 data entries from each dataset to create a balanced dataset
df_pro = df_pro.sample(15000)
df_van = df_van.sample(15000)
# We concat the datasets
df = pd.concat([df_pro,df_van])

# We convert the hour to just be the hour, without the minutes (i.e 22:23 = 22:00)
# and also convert it to the appropriate timezone to match the weather data later
df["Date"] = df.apply(lambda x: pd.to_datetime(x.Date + " " + x.Time).round("H").tz_localize("ETC/GMT-7"), axis = 1)

# We convert the day of the week into an integer
df['DayOfWeek'] = df['DayOfWeek'].map({'Monday':0,'Tuesday':1,'Wednesday':2,'Thursday':3,'Friday':4,'Saturday':5,'Sunday':6})

# We convert the district into an integer 
df['PdDistrict'] = df['PdDistrict'].map({'BAYVIEW':0,'CENTRAL':1,'INGLESIDE':2,'MISSION':3,'NORTHERN':4,'PARK':5,'RICHMOND':6,'SOUTHERN':7,'TARAVAL':8,'TENDERLOIN':9})

# We extract the month from the time data
df['Month'] = pd.to_datetime(df['Date']).dt.month

# We extract the hour from the time data
df['Time'] = pd.to_datetime(df['Time']).dt.hour

# We make the categorical data for prostitution and vandalism into integers
df['Category'] = df['Category'].map({'PROSTITUTION':0,'VANDALISM':1})

# We set the index to the datetime
df.set_index('Date', inplace=True)

# And display the first 10 incidents in the dataframe
df.head(10)

,Category,DayOfWeek,Time,PdDistrict,Month
Date,,,,,
2005-08-20 04:00:00+07:00,0,5,3,4,8
2009-05-06 17:00:00+07:00,0,2,16,9,5
2011-07-31 00:00:00+07:00,0,6,0,3,7
2006-05-23 21:00:00+07:00,0,1,20,7,5
2012-02-28 22:00:00+07:00,0,1,22,9,2
2011-05-18 03:00:00+07:00,0,2,3,4,5
2011-06-09 00:00:00+07:00,0,3,0,3,6
2004-05-19 17:00:00+07:00,0,2,17,3,5
2011-06-23 00:00:00+07:00,0,3,0,3,6


***A training and test dataset is chosen with the use of the sklearn.model_selection package***

In [34]:
# We choose our features
X = df.iloc[:, 1:5].values
# We choose targets
y = df.iloc[:, 0].values

from sklearn.model_selection import train_test_split

# We split it in training and test data with our features and target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=20)

***Now a fit is made on the dataset with a random forest classifier***

In [35]:
from sklearn.ensemble import RandomForestClassifier

# Initialise the randomforestclassifier as classifier, with 20 estimators
classifier = RandomForestClassifier(n_estimators=20, random_state=20)
# We fit the classifier on our training set with features and target
classifier.fit(X_train, y_train)

# We make a prediction on the test set 
y_pred = classifier.predict(X_test)

# We a prediction on the training set
y_train_pred = classifier.predict(X_train)

***Now the training and test set accuracy can be reported as follows:***

In [36]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

#print(confusion_matrix(y_test,y_pred))
#print(classification_report(y_test,y_pred))
print('training set accuracy:')
print(accuracy_score(y_train, y_train_pred))
print('test set accuracy:')
print(accuracy_score(y_test, y_pred))

training set accuracy:
0.8895925925925926
test set accuracy:
0.807


*From the incident reports dataset, the two categories `PROSTITUTION` and `VANDALISM` was chosen. The training and test set was divided at a ratio of 9:1 with no validation set. In order to insure a balanced dataset, 15000 data points were sampled from both datasets. Using a balanced dataset should greatly increase the model prediction accuracy in the future, as it is not skewed by an uneven amount of data. For instance, if we had 15000 datapoints with the `PROSTITUTION` category and 1000 points of the `VANDALISM` category, then the model is much more likely to predict `PROSTITUTION` over `VANDLISM`*

In the above model, it is clear that the training set prediction accuracy is 89% compared to the test set prediction accuracy of 82%. This is of course not a massive descrepancy, but it should be taken into consideration, if the model prediction accuracy should be improved - ideally it will give the same accuracy for the training and test set. The small difference in prediction accuracy may be accredited to the fact that the Random Forest has been used over just a Decision Tree - this is a good way of eliminating the risk of overfitting on datasets, since it will generate several decision trees and then use the model that overfits the least but with the higest prediction accuracy possible. 

In this case, one-hot encoding was not used - categorical data was transformed into integers to allow for easier interpretation of the classifier.

____

### Part 2B: Info from weather features

* Now add features from weather data to your random forest. 
  - Report accuracy. 
  - Discuss how the model performance changes relative to the version with no weather data.
  - Discuss what you have learned about crime from including weather data in your model.

***The weather file is imported and formatted as necessary:***

In [37]:
# The weather file is imported
weather = pd.read_csv("weather.csv", 
                      parse_dates=["date"],
                      date_parser=lambda x: pd.to_datetime(x).tz_convert(None).tz_localize("Etc/GMT+3").tz_convert("Etc/GMT-7"))

# A `Date` column is created
weather['Date'] = weather['date']

# The `date` column is removed
weather = weather.drop('date', 1)

# The Date is converted to the index
weather.set_index('Date', inplace=True)

# The weather data is made categorical
weather['weather'] = pd.Categorical(weather['weather'], categories=weather['weather'].unique()).codes
weather.head()

TypeError: Cannot convert tz-naive Timestamp, use tz_localize to localize

In [ ]:
# The weather data is merged with the incidents dataframe from the previous exercise
df_merge = pd.merge(df, weather, how='inner', left_index=True, right_index=True).dropna()
df_merge.head()

***The features are chosen as X and the target is chosen as y***

In [ ]:
X = df_merge.iloc[:, 1:11].values
y = df_merge.iloc[:, 0].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=20)

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

***The random forest classifier is initialised with 20 trees***

In [ ]:
classifier = RandomForestClassifier(n_estimators=20, random_state=20)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

***The reported prediction accuracy can now be calculated as follows:***

In [ ]:
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

*Above, it was seen that the overall prediction accuracy of the random forest classifier on the test set was 92%. This is means that the weather data has improved the model accuracy with 10% from the previous exercise. Therefore, it goes to show that weather plays an important role in predicting some of the crime patterns that can be seen in San Francisco. Furthermore, from researching this topic online, [it seems that several news journals, like this one](https://www.zmescience.com/science/weather-crime-connection-04234/) report this phenomenon. This information is crucial, in helping law enforcement in dealing with a temperature rise and the following violent acts that follow.*

____

## Part 3: Data visualization

***Exercise 3***

- Create the Bokeh visualization from Part 2 of the Week 8 Lecture, displayed in a beautiful `.gif` below. 
- Provide nice comments for your code. Don't just use the `# inline comments`, but the full Notebook markdown capabilities and explain what you're doing.

![Movie](./bokeh.gif "movie")

***Start off by importing the policing dataframe and parse the date and time as a datetime***

In [ ]:
df = pd.read_csv('../rawdata/Police_Department_Incident_Reports__Historical_2003_to_May_2018.csv', parse_dates=[['Date', 'Time']])
df = df[(df['Date_Time'].dt.year >= 2010) & (df['Date_Time'].dt.year <= 2018)]

*In this case, we want to use the focuscrimes, provided earlier, to make the dataset more managable. These will be the features that we can later select to include or disclude from our bokeh visualisation*

In [ ]:
focuscrimes = ['WEAPON LAWS', 'PROSTITUTION', 'DRIVING UNDER THE INFLUENCE', 'ROBBERY', 'BURGLARY', 'ASSAULT', 'DRUNKENNESS', 'DRUG/NARCOTIC', 'TRESPASS', 'LARCENY/THEFT', 'VANDALISM', 'VEHICLE THEFT', 'STOLEN PROPERTY', 'DISORDERLY CONDUCT']

df_norm = pd.DataFrame(np.zeros((24, len(focuscrimes))),columns=focuscrimes)
df_norm['Hour'] = [x for x in range(1, 25)]

# we iterate over the focuscrimes to add them to the bokeh visualisation
for i, crime_type in enumerate(focuscrimes):
    df_CT = df[df['Category'] == crime_type]
    countByHour_CT = df_CT.groupby(df['Date_Time'].dt.hour).count()["PdId"]
    df_norm[crime_type] = [float(i)/sum(countByHour_CT.values) for i in countByHour_CT.values]
df_norm.head()

*We import the bokeh.io library and check that bokeh is loaded*

In [ ]:
from bokeh.io import output_notebook, show
output_notebook()

*We import the plotting and models methods to use for later plotting*

In [ ]:
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, FactorRange, Legend

source = ColumnDataSource(df_norm)
p = figure(plot_width=1000, plot_height=400, x_range = FactorRange(factors=list(map(str,df_norm['Hour'].values.tolist()))), title="Crimes By Hour", x_axis_label='Hour of The Day', y_axis_label='Relative Frequency')

*X axis is 24-hour which is set by 'FactorRange', a Range of values for a categorical dimesion. Other parameters are configured to set up a Bokeh figure.*

In [ ]:
from bokeh.palettes import Category20

bar = {}
items = []
for indx,(i,color) in enumerate(zip(focuscrimes, Category20[14])):
    bar[i] = p.vbar(x='Hour', top=i, source=source, alpha=1, color=color, width=2, muted_alpha=0.08, muted_color=color, muted=True)
    items.append((i, [bar[i]]))
legend = Legend(items=items, location=(0,-30))

*To construct a bar plot, both x-axis and y-axis is selected from the Dataframe. 'alpha' decides the color depth. 'muted_color' and 'muted_alpha' are set for the interactive mode is 'Muted'.*

In [ ]:
p.add_layout(legend, "right")
p.legend.click_policy="mute"
show(p)

*Legend is configured separately and can be clicked to mute specific bar.*